# This code will train the model and save weights  using a data generator

First Import libraries

In [2]:
import os
import numpy as np
import keras
import tensorflow
from sklearn.model_selection import train_test_split
import nibabel as nib
import tensorflow as tf
from tensorflow.python.keras.utils.data_utils import Sequence
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
from keras_unet.metrics import dice_coef
from keras_unet.models import custom_unet
from keras_unet.losses import jaccard_distance
from sklearn.model_selection import train_test_split
from PIL import Image
from PIL import ImageOps
import fnmatch
import nibabel as nib
import shutil

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


Define our Data Generator and other functions to be used later

In [25]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=12, dim=(512,512,96), n_channels=1,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #X[i,] = np.load('data/' + ID + '.npy')
            X[i,] = np.load('OG-data\\' + ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)



In [9]:
def gather_set(data_path, phrase):
    set_of = []
    path = data_path + '\\'
    for f in os.listdir(data_path):
      if phrase in f:
        set_of.append(f)
      else:
        continue
    #set_of = np.array(set_of)

    indices = np.array(range(len(set_of))) # we will use this in the next step.

    return set_of

Define the paths where images and segmentations are located. Gather into groups and arrange into a dictionary that can be split up for training, testing, and validation for the model

In [10]:
data_path = r"C:\Users\UAB\Kidney-Segmentation-Jupyter\OG-data\\"

images = gather_set(data_path, '_M.')
labels = gather_set(data_path, '_K')
print(images[0:2], labels[0:2])
print(len(images))

['101934_1_96_L_M.npy', '101934_1_96_R_M.npy'] ['101934_1_96_L_M_K.npy', '101934_1_96_R_M_K.npy']
3


In [11]:
d = {}
for i in images:
    if i not in d:
        d[i] = len(d)

labels_mapping = list(map(d.get, images))
print(labels_mapping[0:2])

labels = {images[i]:labels_mapping[i] for i in range(len(images))}
#print(list(labels.items())[:4])



[0, 1]


In [12]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(list(labels.keys()),train_size = 0.8)
partition = {'train':train, 'validation':val}

#print(list(partition.items())[:4])

Create our data generators 

In [19]:
params = {'dim': (512,512,96),
          'batch_size': 12,
          'n_classes': 2,
          'n_channels': 1,
          'shuffle': True}
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [20]:
model = custom_unet(input_shape=(512, 512, 1))


In [21]:
# define the checkpoint
from keras.callbacks import ModelCheckpoint
filepath = "data_gen_test.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [22]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
  loss=jaccard_distance,
  metrics=[dice_coef])

In [26]:
# Train model on dataset
model.fit(training_generator,
                    validation_data = validation_generator,
                    use_multiprocessing=True,
                    workers=6)

['101934_1_96_L_M.npy', '101934_1_96_R_M.npy']


ValueError: could not broadcast input array from shape (512,512,96) into shape (512,512,96,1)